## Optimization TD3 Notebook - Integer Linear Programming

In [25]:
# Introduction

import numpy as np
import pandas as pd
from math import floor
import pulp
from pulp_solving import pulp_solve

# Chargement des données

V = np.loadtxt("VolumeItems50.txt")
N = len(V)       # nombre d'items
C = 2.7          # capacité des boîtes

In [26]:
# Création de la liste des boites

Indices3 = np.where(V <= C/3)[0]
Indices2 = np.where((V > C/3) & (V <= C/2))[0]
Indices1 = np.where(V > C/2)[0]

Box = []

# Boîtes contenant 3 items
nB3 = floor(len(Indices3) / 3)
for b in range(nB3):
    items = Indices3[3*b:3*(b+1)]
    box = {
        'NumberBox': b+1,
        'Items': items+1,
        'NumberItems': len(items),
        'UnusedVolume': C - np.sum(V[items])
    }
    Box.append(box)

# Items restants du groupe 3
nI3 = len(Indices3) % 3
if nI3 > 0:
    Indices2 = np.concatenate([Indices2, Indices3[-nI3:]])

# Boîtes contenant 2 items
nB2 = floor(len(Indices2) / 2)
for b in range(nB2):
    noBox = nB3 + b + 1
    items = Indices2[2*b:2*(b+1)]
    box = {
        'NumberBox': noBox,
        'Items': items+1,
        'NumberItems': len(items),
        'UnusedVolume': C - np.sum(V[items])
    }
    Box.append(box)

# Items restants du groupe 2
nI2 = len(Indices2) % 2
if nI2 > 0:
    Indices1 = np.concatenate([Indices1, Indices2[-nI2:]])
    
# Boîtes contenant 1 item
nB1 = len(Indices1)
for b in range(nB1):
    noBox = nB3 + nB2 + b + 1
    items = np.array([Indices1[b]])
    box = {
        'NumberBox': noBox,
        'Items': items+1,
        'NumberItems': len(items),
        'UnusedVolume': C - np.sum(V[items])
    }
    Box.append(box)

total_boxes = nB3 + nB2 + nB1
df = pd.DataFrame(data = Box)

# Affichage des résultats
print(df)
print("\nNumber of boxes:", total_boxes)

    NumberBox         Items  NumberItems  UnusedVolume
0           1     [2, 3, 4]            3          1.35
1           2    [5, 8, 13]            3          0.98
2           3  [14, 15, 16]            3          0.99
3           4  [18, 21, 22]            3          1.92
4           5  [24, 25, 26]            3          1.58
5           6  [27, 30, 31]            3          1.06
6           7  [32, 34, 37]            3          1.44
7           8  [38, 39, 40]            3          0.63
8           9  [42, 43, 45]            3          1.84
9          10  [46, 49, 50]            3          0.81
10         11      [10, 19]            2          0.61
11         12      [23, 36]            2          0.74
12         13      [41, 48]            2          0.44
13         14           [1]            1          1.19
14         15           [6]            1          0.48
15         16           [7]            1          0.90
16         17           [9]            1          0.99
17        

In [27]:
# [Question 1] - Remplissage séquentiel des items (heuristique)

# N est le nombre d'items, V est le vecteur des volumes de ces items.
# On fait une boucle sur N: pour chaque item, on regarde d'abord si il reste de
# la place dans la boite précédente (si volume S < capacité C). Si oui, on la 
# remplit et on met à jour son volume (S = S + V(i)). Si non, on prend une nouvelle
# boite et on la remplit. Ainsi de suite jusqu'à avoir paqueté tous les items.

nb_box = 0         # Indice de la boîte utilisée
S = 0              # Volume de la boite actuelle (en cours de remplissage)
i = 0              # Indice de l'objet en cours de paquetage
Box_seq = []       # liste des boîtes

Box_seq.append({'NumberBox': 1, 'Items': []})

while i < N:
    if S + V[i] <= C:
        S += V[i]
        Box_seq[nb_box]['Items'].append(i)
        i += 1
    else:
        Box_seq[nb_box]['NumberItems'] = len(Box_seq[nb_box]['Items'])
        Box_seq[nb_box]['UnusedVolume'] = C - S
        # Ouvrir une nouvelle boîte
        nb_box += 1
        Box_seq.append({'NumberBox': nb_box+1, 'Items': []})
        S = 0

# Finalisation de la dernière boîte
if 'NumberItems' not in Box_seq[nb_box]:
    Box_seq[nb_box]['NumberItems'] = len(Box_seq[nb_box]['Items'])
    Box_seq[nb_box]['UnusedVolume'] = C - S

print(pd.DataFrame(data=Box_seq))
print("\nNumber of used boxes (greedy heuristic) :", len(Box_seq))

    NumberBox             Items  NumberItems  UnusedVolume
0           1         [0, 1, 2]            3          0.01
1           2         [3, 4, 5]            3          0.01
2           3            [6, 7]            2          0.01
3           4            [8, 9]            2          0.01
4           5              [10]            1          1.12
5           6          [11, 12]            2          0.01
6           7      [13, 14, 15]            3          0.99
7           8              [16]            1          0.45
8           9          [17, 18]            2          1.13
9          10      [19, 20, 21]            3          0.29
10         11  [22, 23, 24, 25]            4          0.62
11         12          [26, 27]            2          0.47
12         13          [28, 29]            2          0.13
13         14      [30, 31, 32]            3          0.16
14         15              [33]            1          2.45
15         16              [34]            1          0.

In [28]:
# [Question 2]

# On a un problème linéaire entier (ILP). L'ensemble N des objets est fini,
# on peut donc redéfinir le problème comme binary linear problem, en
# introduisant des variables binaires.

On définit les variables suivantes:

$$
x_{n,b} =
\begin{cases} 
1, & \text{si l'objet } n \text{ est placé dans la boîte } b \\
0, & \text{sinon}
\end{cases}
\hspace{2cm}

y_b =
\begin{cases} 
1, & \text{si la boîte } b \text{ est utilisée} \\
0, & \text{sinon}
\end{cases}
$$

On définit également:

$$
\begin{cases} 
v_n & \text{est le volume de l'objet } n,\\
C & \text{est la capacité maximale d'une boîte,}\\
N & \text{est le nombre total d'objets,}\\
B & \text{est la borne supérieure du nombre de boîtes obtenue dans la question 1.}
\end{cases}
$$

On peut donc écrire les contraines et le coût ainsi:

$$
\textbf{C1: Le volume des boites ne doit pas dépasser leur capacité} \\
\text{Pour chaque boite, on somme sur les volumes des objets présents et on vérifie que cela est inférieur à la capacité C}
$$

$$
\forall b \in \{1, \dots, B\}, \quad \sum_{1 \leq n \leq N} v_n x_{n,b} \leq C y_b
$$

$$
\textbf{C2: Tous les items doivent être dans une boite} \\
\text{Pour chaque item, on vérifie qu'il est au moins placé dans une des boites} 
$$

$$
\forall n \in \{1, \dots, N\}, \quad \sum_{b=1}^{B} x_{n,b} = 1
$$

$$
\textbf{Fonction de coût, à minimiser}
$$

$$
\min \sum_{b=1}^{B} y_b
$$

On a donc finalement le résultat suivant:

$$
\alpha_n = v_n \hspace{1cm} \beta = C
$$

In [29]:
# [Question 3]

# On veut reformuler le problème comme un problème ILP

Pour simplifier (et car Matlab nous permet de le calculer ainsi), on garde les contraintes d'égalité séparées.
Pour cela, on doit donc écrire le problème comme:

$$
\text{Minimise} \ \langle C, X \rangle
\hspace{1cm}
\text{tel que} 
\begin{cases}
AX \leq B \\
A_{eq}X = B_{eq}
\end{cases}
\hspace{1cm}
\text{où} \ X \in \{0,1\}^{(N+1)*B}
$$

On cherche donc à écrire les matrices $A, B, A_{eq}, B_{eq}, C \ \text{et} \ X$ adéquates (formalisme mathématique). On remarque que la fonction de coût ne fait pas intervenir explicitement la variable x précédemment définie, on va donc créer un nouveau vecteur X, qui contiendra une représentation en ligne de nos $x_{i,j}$, ainsi que les $y_k$

$$
X = 
\begin{bmatrix}
x_{1,1} & x_{1,2} & \dots & x_{1,B} \dots x_{N,1} & x_{N,2} & \dots & x_{N,B} \dots y_1 & y_2 & \dots & y_B
\end{bmatrix}^\top
$$

Pour simplifier, on explicite les matrices suivantes dans le code, de telle sorte à conserver les contraintes C1, C2 et l'objectif O1.

In [30]:
# Implémentation des 4 vecteurs / matrices précédemment définies

# Borne supérieure calculée lors de la question 1
B_bound = 21

# --- Matrice C pour la fonction de coût ---
# On ne s'interesse qu'aux variables y_i, en bout de vecteur. 
# On remplit donc de 1 sur les dernières B valeurs du vecteur.
C_matrix = np.concatenate([np.zeros(B_bound * N), np.ones(B_bound)])

# --- Matrice A et B pour la contrainte d'inégalité ---
# La matrice A correspond à la juxtaposition horizontale de matrices
# V(i)*Id_B + une matrice -C*Id_B (soustraction de la borne de capacité de chaque boite)
# La matrice B est un vecteur de zeros.
M_list = [V[i] * np.eye(B_bound) for i in range(N)]
M_1 = np.hstack(M_list)
M_2 = -C * np.eye(B_bound)
A_matrix = np.hstack([M_1, M_2])
B_matrix = np.zeros(B_bound)

# --- Matrice Aeq et vecteur Beq pour les contraintes d'égalité ---
# La matrice Aeq correspond à la juxtaposition horizontale de vecteurs ligne de
# dimension B contenant uniquement des 1. A chaque juxtaposition, on
# descend d'une ligne, et ce sur les N lignes. On juxtapose une matrice
# (N,B) de zeros à la fin (car les variables y_i ne font pas partie de la contrainte).
# La matrice B_eq est un vecteur de 1.
Aeq_matrix = np.zeros((N, B_bound * N + B_bound))
for i in range(N):
    Aeq_matrix[i, i * B_bound:(i + 1) * B_bound] = 1
Beq_matrix = np.ones(N)

print("Dimensions de A_matrix :", A_matrix.shape)
print("Dimensions de Aeq_matrix :", Aeq_matrix.shape)

Dimensions de A_matrix : (21, 1071)
Dimensions de Aeq_matrix : (50, 1071)


In [31]:
# Résolution numérique du problème avec Pulp

x_vars, y_vars = pulp_solve(C, A_matrix, Aeq_matrix, B_matrix, Beq_matrix, B_bound, N)

Problème ILP construit. Résolution en cours...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/alexandregravereaux/.pyenv/versions/3.10.12/envs/test/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/gq/r812n66d5695j67xqqps21t00000gn/T/bc16574f91874af494db2909d80be1f3-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/gq/r812n66d5695j67xqqps21t00000gn/T/bc16574f91874af494db2909d80be1f3-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 76 COLUMNS
At line 5411 RHS
At line 5483 BOUNDS
At line 6555 ENDATA
Problem MODEL has 71 rows, 1071 columns and 2121 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 182.06 - 0.00 seconds
Cgl0003I 0 fixed, 21 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 71 rows, 1071 columns (1071 integer (1050 of which bina

In [32]:
# Affichage des résultats

nb_box_ilp = sum([pulp.value(var) for var in y_vars])
print("Nombre de boîtes (ILP) :", nb_box_ilp)

Nombre de boîtes (ILP) : 18.0


In [10]:
# [Question 4]

# La contrainte C3 nous impose d'insérer les objets du groupe 1 dans au plus de 2 boites.
# Comme les boites ont la meme capacité volumique et comme l'ordre de placement des items
# dans les boîtes n'est pas imposé nous pouvons considérer, sans perte de généralité,
# que nous placons ces items dans les boites 1 et 2 (quitte à rééchanger). Cette contrainte se formule ainsi:

C3: Les items du groupe G1 = {1, 2, 3, 4, 5} doivent être placés dans au plus deux boîtes.

$$
\sum_{b=1}^{2} \sum_{n=1}^{5} x_{n,b} = 5
$$

In [33]:
# Construction des nouvelles matrices

# On rajoute une ligne à la matrice Aeq avec uniquement des 1 sur les 2B
# premières colonnes, puis uniquement des 0. De même, on rajoute un 5 au
# bout du vecteur Beq (désormais de taille N+1)

Box_line = np.array([1, 1] + [0] * (B_bound - 2))
Aeq_add = np.array([])

for _ in range(5):
    Aeq_add = np.concatenate([Aeq_add, Box_line])

Aeq_add = np.concatenate([Aeq_add, np.zeros(46 * B_bound)])
Aeq_add = Aeq_add.reshape((1, Aeq_add.shape[0]))

Aeq_matrix_2 = np.concatenate([Aeq_matrix, Aeq_add])
Beq_matrix_2 = np.concatenate([Beq_matrix, np.array([5])])

# Resolution numérique

x_vars_2, y_vars_2 = pulp_solve(C, A_matrix, Aeq_matrix_2, B_matrix, Beq_matrix_2, B_bound, N)


Problème ILP construit. Résolution en cours...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/alexandregravereaux/.pyenv/versions/3.10.12/envs/test/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/gq/r812n66d5695j67xqqps21t00000gn/T/5a15ae62c40e414eb63c4c16de099765-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/gq/r812n66d5695j67xqqps21t00000gn/T/5a15ae62c40e414eb63c4c16de099765-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 76 COLUMNS
At line 5411 RHS
At line 5483 BOUNDS
At line 6555 ENDATA
Problem MODEL has 71 rows, 1071 columns and 2121 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 182.06 - 0.00 seconds
Cgl0003I 0 fixed, 21 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 71 rows, 1071 columns (1071 integer (1050 of which bina

In [34]:
# Affichage des résultats

nb_box_ilp_2 = sum([pulp.value(var) for var in y_vars_2])
print("Nombre de boîtes (ILP) :", nb_box_ilp_2)

Nombre de boîtes (ILP) : 18.0


In [ ]:
# [Question 5]

Pour la contrainte C4, on aimerait choisir les indices comme la C3 sauf que les objets du groupe 2 peuvent partager les boites avec le groupe 1 donc on ne peut plus choisir les boites.

En effet, plusieurs cas se présentent:
- Soit on les ajoute dans des nouvelles boites non utilisées par le groupe 1
- Soit on ajoute certains items dans l'espace restant des boites utilisées par le groupe 1 (boite 1 et/ou boite 2).

Nous ne pouvons pas savoir quel cas se présente et cela nous empêche donc de choisir les boites à l'avance comme dans la question précédente.
Par contre ce qu'on peut faire c'est indiquer qu'on les mets dans 3 boites cote a cote (quitte a echanger les boites), donc ce qu'on peut faire c'est prendre 2 indices au hasard dans le groupe 2 et regarder si ils sont dans des boites avec une distance indicielle maximum de 2.

$\sum_{1 \leq b \leq B} b \cdot x_{n,b}$ représente l'indice de la boite ou l'objet n est placé

On peut donc ecrire la contrainte C4 ainsi:

$
\forall (i,j) \in [6,11], \sum_{1 \leq b \leq B} b \cdot x_{i,b} - \sum_{1 \leq b \leq B} b \cdot x_{j,b} <=2
$

In [35]:
# Construction des nouvelles matrices A et B, pour la contrainte d'inégalité

len_boites = 5
A_matrix_2 = A_matrix.copy()

for k in range(len_boites):
    A_add = np.zeros((len_boites, (N + 1) * B_bound))

    for b in range(B_bound):
        ones_col = b * np.ones((len_boites, 1))
        neg_identity = -b * np.eye(len_boites)
        M_inject = np.hstack((ones_col, neg_identity))

        start_col = (k) + 5 + (b * B_bound)
        end_col = start_col + 6
        A_add[:, start_col:end_col] = M_inject

    # On ajoute une 2e fois -A_add avec le signe négatif pour avoir l'inégalité 
    # dans l'autre sens et ainsi simuler une valeur absolue.
    A_matrix_2 = np.vstack((A_matrix_2, A_add, -A_add))

B_matrix_2 = np.hstack((B_matrix, np.ones(((len_boites ** 2) *2,))))

# On a bien des matrices de taille 71 = B + 2*len_boites^2 (pour le nb de lignes)
print(A_matrix_2.shape)
print(B_matrix_2.shape)

# Resolution numérique

x_vars_3, y_vars_3 = pulp_solve(C, A_matrix_2, Aeq_matrix_2, B_matrix_2, Beq_matrix_2, B_bound, N)


(71, 1071)
(71,)
Problème ILP construit. Résolution en cours...
Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/alexandregravereaux/.pyenv/versions/3.10.12/envs/test/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/i64/cbc /var/folders/gq/r812n66d5695j67xqqps21t00000gn/T/2c1f2bba407241eebdfa3fbe601fef67-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/gq/r812n66d5695j67xqqps21t00000gn/T/2c1f2bba407241eebdfa3fbe601fef67-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 76 COLUMNS
At line 5411 RHS
At line 5483 BOUNDS
At line 6555 ENDATA
Problem MODEL has 71 rows, 1071 columns and 2121 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 182.06 - 0.00 seconds
Cgl0003I 0 fixed, 21 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 71 rows, 1071 columns (1071 integer (1

In [36]:
# Affichage des résultats

nb_box_ilp_3 = sum([pulp.value(var) for var in y_vars_3])
print("Nombre de boîtes (ILP) :", nb_box_ilp_3)

Nombre de boîtes (ILP) : 18.0


[Question 6]

Q6: Comme nous avons résolu un problème ILP, la solution appartient à un ensemble non convexe et n'assure donc pas l'unicité de la solution. Une stratégie courante consiste à appliquer la relaxation continue, c'est-à-dire résoudre le problème dans un espace réel et convexe. Si le même minimiseur est trouvé que dans l'espace d'origine (discret), alors la solution est unique. Sinon, il existe plusieurs solutions distinctes.

[Question 7]

On considère maintenant un problème mixte à 2 objectifs, le 2e objectif s'ecrivant

$
O2 : \max \max_{1 \leq b \leq B} \left( C \cdot y_b - \sum_{1 \leq n \leq N} x_{n,b} v_n \right) = \min_{1 \leq b \leq B} \left( \sum_{1 \leq n \leq N} x_{n,b} v_n \right)
$

On peut donc considérer le MILP suivant

$
\text{minimize} \quad <c_1 | x_1> + < c_2 | x_2 > \quad \text{subject to} \quad L_1 x_1 + L_2 x_2 \geq b
$

Non traité pour l'instant